In [44]:
# セル1: 共通インスタンスの定義に上書き修正

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from janome.tokenizer import Tokenizer

# 形態素解析の初期化
tokenizer = Tokenizer()

# --- 列名定義 (CSVに合わせて、定数を定義) ---
CATEGORY_COL = 'category'
RATING_COL = 'star_rating'
REVIEW_COL = 'review_text'


# OneHotEncoderのインスタンス
OHE_MODEL = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# def encode_categoryの引数を修正
def encode_category(df, column_name=CATEGORY_COL, fit_mode=False):
    """カテゴリ列をワンホットエンコーディングする。"""
    categories = df[[column_name]]
    
    if fit_mode:
        return OHE_MODEL.fit_transform(categories)
    else:
        return OHE_MODEL.transform(categories)

# MinMaxScalerのインスタンス
SCALER_MODEL = MinMaxScaler(feature_range=(0, 1))

# def scale_ratingの引数を修正
def scale_rating(df, column_name=RATING_COL, fit_mode=False):
    """星の数 (1.0~5.0) を0~1に正規化する。"""
    # RATING_COLを使うように修正
    ratings = df[[column_name]]
    
    if fit_mode:
        return SCALER_MODEL.fit_transform(ratings)
    else:
        return SCALER_MODEL.transform(ratings)

# TfidfVectorizerのインスタンス
TFIDF_MODEL = TfidfVectorizer(max_features=5000) 

# def vectorize_reviewの引数を修正
def preprocess_review(text):
    """レビューテキストを形態素解析し、スペース区切り文字列に変換する前処理"""
    if pd.isna(text): return ""
    tokens = [token.surface for token in tokenizer.tokenize(text) if token.part_of_speech.split(',')[0] in ['名詞', '形容詞']]
    return " ".join(tokens)

def vectorize_review(df, column_name=REVIEW_COL, fit_mode=False):
    """レビュー列をTF-IDFベクトルに変換する。"""
    # REVIEW_COLを使うように修正
    processed_reviews = df[column_name].apply(preprocess_review)
    
    if fit_mode:
        return TFIDF_MODEL.fit_transform(processed_reviews).toarray()
    else:
        return TFIDF_MODEL.transform(processed_reviews).toarray()
    


# --- ユーザーベクトル作成のための列名修正 ---
# 既存の関数定義のセルに戻り、以下の2行を上部に追記してください。
CATEGORY_COL = 'category' 
RATING_COL = 'star_rating' 
REVIEW_COL = 'review_text'

# 既存の関数定義の引数 'ジャンル' や '平均評価' を、上記定数を使うように修正します。
# 例: 


# ※この修正を既存のコードブロックに適用してください。

## pattern_total3


In [45]:
# --- 特徴量行列の作成 (店舗データの特徴量生成) ---

def create_pattern1_features(df): #パターン1
    """
    パターン1: カテゴリ + 星の数 の特徴量行列を生成する
    """
    # column_nameの指定を削除し、定数を使う
    category_features = encode_category(df, fit_mode=True) 
    rating_features = scale_rating(df, fit_mode=True)
    final_features = np.concatenate([category_features, rating_features], axis=1)
    return final_features

def create_pattern2_features(df): #パターン２
    # column_nameの指定を削除
    category_features = encode_category(df, fit_mode=True)
    review_features = vectorize_review(df, fit_mode=True)
    final_features = np.concatenate([category_features, review_features], axis=1)
    return final_features

def create_pattern3_features(df): #パターン３
    # column_nameの指定を削除
    category_features = encode_category(df, fit_mode=True)
    rating_features = scale_rating(df, fit_mode=True)
    review_features = vectorize_review(df, fit_mode=True)
    final_features = np.concatenate([category_features, rating_features, review_features], axis=1)
    return final_features

## ユーザーベクトル

In [46]:
# --- ユーザーベクトル ---

def create_user_vector_pattern1(category_query, rating_query):
    # P1: カテゴリ + 星の数
    category_df = pd.DataFrame([category_query], columns=[CATEGORY_COL])
    user_category_vector = encode_category(category_df, fit_mode=False)
    
    rating_df = pd.DataFrame([rating_query], columns=[RATING_COL])
    user_rating_vector = scale_rating(rating_df, fit_mode=False)
    
    user_final_vector = np.concatenate([user_category_vector, user_rating_vector], axis=1)
    return user_final_vector.flatten()


def create_user_vector_pattern2(category_query, review_query):
    # P2: カテゴリ + レビュー
    category_df = pd.DataFrame([category_query], columns=[CATEGORY_COL])
    user_category_vector = encode_category(category_df, fit_mode=False)
    
    review_df = pd.DataFrame([review_query], columns=[REVIEW_COL])
    user_review_vector = vectorize_review(review_df, fit_mode=False)
    
    user_final_vector = np.concatenate([user_category_vector, user_review_vector], axis=1)
    return user_final_vector.flatten()


def create_user_vector_pattern3(category_query, review_query, rating_query):
    # P3: カテゴリ + レビュー + 星の数
    category_df = pd.DataFrame([category_query], columns=[CATEGORY_COL])
    user_category_vector = encode_category(category_df, fit_mode=False)
    
    review_df = pd.DataFrame([review_query], columns=[REVIEW_COL])
    user_review_vector = vectorize_review(review_df, fit_mode=False)
    
    rating_df = pd.DataFrame([rating_query], columns=[RATING_COL])
    user_rating_vector = scale_rating(rating_df, fit_mode=False)
    
    user_final_vector = np.concatenate([user_category_vector, user_review_vector, user_rating_vector], axis=1)
    return user_final_vector.flatten()


In [47]:
# --- データ読み込みと前処理の実行 ---

# TODO: 1. CSVファイル
CSV_FILE_PATH = '/Users/dangararara/lecture/miraisouzou/suku/odaiba_reviews_4.csv' 

# データの読み込み
try:
    # エンコーディングはデータに合わせて 'utf-8' や 'shift_jis' を試してください
    shop_df = pd.read_csv(CSV_FILE_PATH, encoding='utf-8')
    print(f"CSVファイル '{CSV_FILE_PATH}' を読み込みました。")
except Exception as e:
    print(f"CSVの読み込み中にエラーが発生しました ({e})。ファイル名とエンコーディングを確認してください。")
    # ここで処理を中断する必要がある場合は sys.exit() などを検討してください。

# データのクレンジングと型変換
# star_rating列を数値型に変換
shop_df[RATING_COL] = pd.to_numeric(shop_df[RATING_COL], errors='coerce') 
# 必須項目がない行は除外
shop_df.dropna(subset=[RATING_COL, CATEGORY_COL, REVIEW_COL], inplace=True) 

print("データ準備完了。")
print(shop_df[[CATEGORY_COL, RATING_COL]].head())
print(f"総店舗数: {len(shop_df)}")
print("-" * 30)

CSVファイル '/Users/dangararara/lecture/miraisouzou/suku/odaiba_reviews_4.csv' を読み込みました。
データ準備完了。
             category  star_rating
0  台場駅,東京,海鮮,日本料理,居酒屋         3.33
1  台場駅,東京,海鮮,日本料理,居酒屋         3.33
2  台場駅,東京,海鮮,日本料理,居酒屋         3.33
3  台場駅,東京,海鮮,日本料理,居酒屋         3.33
4  台場駅,東京,海鮮,日本料理,居酒屋         3.33
総店舗数: 29385
------------------------------


##  お店ごとのレビュー統合

In [48]:
# --- データ集約（グルーピング）の実行 ---
# 新しいセルとして追加してください

# グループ化のキー (店舗IDの代わりに使用)
GROUP_KEY = 'shop_name'

# 1. 各店舗の平均評価を計算
avg_ratings = shop_df.groupby(GROUP_KEY)[RATING_COL].mean().reset_index()
avg_ratings.rename(columns={RATING_COL: 'aggregated_rating'}, inplace=True)

# 2. 全レビュー本文を店舗ごとに結合
# NaNやNoneを除外してから結合します
review_texts = shop_df.groupby(GROUP_KEY)[REVIEW_COL].apply(
    lambda x: ' '.join(x.dropna().astype(str))
).reset_index()
review_texts.rename(columns={REVIEW_COL: 'aggregated_review_text'}, inplace=True)


# 3. カテゴリやURLなどの静的な情報を取得 (最初の値を使う)
static_info = shop_df.drop_duplicates(subset=[GROUP_KEY]).reset_index(drop=True)
static_info = static_info.drop(columns=[RATING_COL, REVIEW_COL])


# 4. 全ての集約結果を結合して新しいデータフレームを作成
grouped_df = pd.merge(static_info, avg_ratings, on=GROUP_KEY, how='left')
grouped_df = pd.merge(grouped_df, review_texts, on=GROUP_KEY, how='left')


# 以降の処理のために、元の列名に戻す
grouped_df[RATING_COL] = grouped_df['aggregated_rating']
grouped_df[REVIEW_COL] = grouped_df['aggregated_review_text']


# 今後の実験で使うデータフレームを上書き
shop_df = grouped_df

print("✅ データ集約が完了しました。")
print(f"集約後の店舗数: {len(shop_df)}")
print("-" * 30)

# --- ユーザー希望シナリオの再定義 ---
# このセルで user_rating_min の値を再設定します
user_rating_min = 4.0

✅ データ集約が完了しました。
集約後の店舗数: 404
------------------------------


## ユーザーの入力

In [49]:
# --- ユーザー希望（実験シナリオ）の設定 ---
# 新しいセルとして追加してください

# シナリオ: 「静かで薄味の和食で、評価4.0以上の場所」を探す
user_category = 'イタリアン'
user_review = 'クリーミなパスタが食べたい。'
user_rating_min = 4.0  # 希望の最低評価 (0.0~5.0)

print(f"ユーザー希望シナリオを設定しました:")
print(f"カテゴリ: '{user_category}'")
print(f"イメージ: '{user_review}'")
print(f"評価: {user_rating_min}以上")
print("-" * 30)

ユーザー希望シナリオを設定しました:
カテゴリ: 'イタリアン'
イメージ: 'クリーミなパスタが食べたい。'
評価: 4.0以上
------------------------------


## レコメンド

In [50]:
# --- 推薦ロジック（コサイン類似度）---

def recommend_shops(shop_df, feature_matrix, user_vector, k=3):
    """
    コサイン類似度で類似度の高い店舗を推薦する。
    """
    from sklearn.metrics.pairwise import cosine_similarity
    
    if user_vector.ndim == 1:
        user_vector = user_vector.reshape(1, -1)
        
    similarities = cosine_similarity(user_vector, feature_matrix)
    top_indices = np.argsort(similarities[0])[-k:][::-1]
    
    recommendations = shop_df.iloc[top_indices].copy()
    recommendations['類似度スコア'] = similarities[0][top_indices]
    
    return recommendations


In [51]:
# --- P1: パターン1 (カテゴリ + 星の数) の実験と結果 ---
# 新しいセルとして追加してください

print("【P1: カテゴリ + 星の数】モデルを学習し、推薦を実行します")
# モデル学習 (特徴量行列作成)
feature_matrix_p1 = create_pattern1_features(shop_df)
print(f"特徴量次元数: {feature_matrix_p1.shape[1]}")

# ユーザーベクトル生成
user_vector_p1 = create_user_vector_pattern1(user_category, user_rating_min)

# 推薦実行
recs_p1 = recommend_shops(shop_df, feature_matrix_p1, user_vector_p1, k=3)

print("\n--- P1 推薦結果 ---")
print(recs_p1[['shop_name', RATING_COL, '類似度スコア']])
print("-" * 30)

【P1: カテゴリ + 星の数】モデルを学習し、推薦を実行します
特徴量次元数: 334

--- P1 推薦結果 ---
           shop_name  star_rating    類似度スコア
113       三代目 博多 だるま         3.67  0.707107
79   RISTORANTE OZIO         3.61  0.673222
221          日本料理　時宜         3.60  0.667120
------------------------------


In [52]:
#--- p2: パターン2 (カテゴリ + レビュー) の実験と結果 ---
# 新しいセルとして追加してください

print("【P2: カテゴリ + レビュー】モデルを学習し、推薦を実行します")
# モデル学習 (特徴量行列作成)
feature_matrix_p2 = create_pattern2_features(shop_df)
print(f"特徴量次元数: {feature_matrix_p2.shape[1]}")

# ユーザーベクトル生成
user_vector_p2 = create_user_vector_pattern2(user_category, user_review)

# 推薦実行
recs_p2 = recommend_shops(shop_df, feature_matrix_p2, user_vector_p2, k=3) 

print("\n--- P2 推薦結果 ---")
print(recs_p2[['shop_name', RATING_COL, '類似度スコア']])
print("-" * 30)

【P2: カテゴリ + レビュー】モデルを学習し、推薦を実行します
特徴量次元数: 5333

--- P2 推薦結果 ---
         shop_name  star_rating    類似度スコア
163    こなな 有明ガーデン店         3.10  0.409517
286  イタリアン酒場 パステーカ         3.08  0.373975
41   カフェ ラ・ボエム お台場         3.31  0.263337
------------------------------


In [53]:
# ---p3: パターン3 (カテゴリ + レビュー + 星の数) の実験と結果 ---
# 新しいセルとして追加してください          


print("【P3: カテゴリ + レビュー + 星の数】モデルを学習し、推薦を実行します")
# モデル学習 (特徴量行列作成)
feature_matrix_p3 = create_pattern3_features(shop_df)
print(f"特徴量次元数: {feature_matrix_p3.shape[1]}")

# ユーザーベクトル生成
user_vector_p3 = create_user_vector_pattern3(user_category, user_review, user_rating_min)

# 推薦実行
recs_p3 = recommend_shops(shop_df, feature_matrix_p3, user_vector_p3, k=3)

print("\n--- P3 推薦結果 ---")
print(recs_p3[['shop_name', RATING_COL, '類似度スコア']])
print("-" * 30)


【P3: カテゴリ + レビュー + 星の数】モデルを学習し、推薦を実行します
特徴量次元数: 5334

--- P3 推薦結果 ---
          shop_name  star_rating    類似度スコア
38      プロント ワンザ有明店         3.07  0.050878
184  CoCo壱番屋 ワンザ有明店         3.05  0.029788
71           鮒忠 有明店         3.08  0.025884
------------------------------
